<a href="https://colab.research.google.com/github/colabllmdi/travel_bot/blob/main/TravelBot_Urgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers langchain accelerate chardet cchardet torch einops accelerate SentencePiece sentence_transformers pinecone-client datasets preprocess gradio xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 61.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Sep 11 10:44:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    47W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import langchain
import os
import gradio as gr
import pandas as pd
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import pinecone
import time
from langchain.chains import ConversationalRetrievalChain, ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import Pinecone

In [ ]:
model_id_llama_2 = "/content/drive/MyDrive/LLM_Models/Llama-2-7b-chat-hf/"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id_llama_2)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    n_gpu = torch.cuda.device_count()
    torch.cuda.get_device_name(0)
else:
    device = torch.device("cpu")

In [ ]:
llama_2_model = AutoModelForCausalLM.from_pretrained(model_id_llama_2).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
embed_model_id = '/content/drive/MyDrive/all-MiniLM-L6-v2'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)
embed_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='/content/drive/MyDrive/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={'device': device(type='cuda')}, encode_kwargs={'device': device(type='cuda'), 'batch_size': 32}, multi_process=False)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/MoT Data Sample/Riyadh_Statement_RAG.csv")
df

,Unnamed: 0,Statement
0,0,"Hotel: Madareem Crown Hotelis located at حي, N..."
1,1,Hotel: Narcissus Hotel Riyadhis located at Ola...
2,2,Hotel: Boudl Al Sahafa Hotelis located at RJ6W...
3,3,Hotel: Novotel Riyadh Al Anoudis located at 90...
4,4,"Hotel: Radisson Blu Hotel, Riyadh Convention &..."
...,...,...
155,155,Place of Tourist Attraction: محمد احمد وحيد نا...
156,156,Place of Tourist Attraction: البيبي ريزيدنسis ...
157,157,Place of Tourist Attraction: نافورة الجامعةis ...
158,158,Place of Tourist Attraction: BurritoMines.mine...


In [ ]:
docs = [
    "/content/drive/MyDrive/MoT Data Sample/Riyadh_Statement_RAG.csv",
]

embeddings = embed_model.embed_documents(docs)

print(f"We have {len(embeddings)} doc embeddings, each with "
      f"a dimensionality of {len(embeddings[0])}.")

We have 1 doc embeddings, each with a dimensionality of 384.


In [ ]:
PINECONE_API_KEY = '844d74e0-7369-4cf4-9e85-18c9d6bc5b4a'

In [ ]:
# get API key from app.pinecone.io and environment from console
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY') or '844d74e0-7369-4cf4-9e85-18c9d6bc5b4a',
    environment=os.environ.get('PINECONE_ENVIRONMENT') or 'us-west1-gcp-free'
)

In [ ]:
index_name = 'llama-2-rag-mot2'

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=len(embeddings[0]),
        metric='cosine'
    )
    # wait for index to finish initialization
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

In [ ]:
index = pinecone.Index(index_name)
index

In [ ]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 160}},
 'total_vector_count': 160}

In [ ]:
time.sleep(10)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 160}},
 'total_vector_count': 160}

In [ ]:
df.columns

Index(['Unnamed: 0', 'Statement'], dtype='object')

In [ ]:
df.columns = ['id','Data']

In [ ]:
data = df.copy()

batch_size = 32

for i in range(0, len(data), batch_size):
    i_end = min(len(data), i+batch_size)
    batch = data.iloc[i:i_end]
    ids = [f"{x['id']}" for i, x in batch.iterrows()]
    texts = [x['Data'] for i, x in batch.iterrows()]
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['Data']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

In [ ]:
SYSTEM_PROMPT = '''

You are a personalized travel assistant.

Assistant should be able to answer questions based on provided context.

Assistant is an experienced trip planner specializing in creating unique and memorable iteneraries based on available data only.

If asked for an itinerary or trip planning, the Assistant will create an itinerary which will include a hotel stay for the entire trip duration and visits to minimum two restaurants and one tourist attraction daily.

Trip itinerary should include breakfast, lunch and dinner recommendations as well.

Assistant will optimise the travel itinerary in a way that hotel stay is for the entire trip duration and rest of the info is according to the location of hotel.

Assistant is able to provide optimized trips based on best available ratings for hotel, restaurant and places of tourist attraction.

Assistant will provide the output in a bullets with one row for each day of the itinerary.

If asked for information, assistant will provide detailed information about on the question asked, based on the available context only.

The answers should be consise and to the point.

'''

In [ ]:

# If asked for an itinerary or trip planning, only then Assistant will create itinerary which will include a hotel stay for the entire trip duration and visits to minimum two restaurants and one tourist attraction daily.

# Assistant will optimise the travel itinerary in a way that hotel stay is for the entire trip duration and rest of the info is according to the location of hotel.

# Assistant will provide the output in a tabular format with one row for each day of the itinerary.

# Assistant will also ensure the table has the following columns: Day, Recommended itinerary, Approximate cost per person.

In [ ]:
def generate_prompt(prompt: str, system_prompt: str = SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

In [ ]:
from langchain import HuggingFacePipeline, PromptTemplate

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
generate_text = transformers.pipeline(
    model=llama_2_model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1,
    device=device# without this output begins repeating
)

In [ ]:
text_field = 'text'  # field in metadata that contains text content

vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)


/usr/local/lib/python3.10/dist-packages/langchain/vectorstores/pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [ ]:
from langchain.chains import RetrievalQA
llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    memory = conversational_memory,
    chain_type_kwargs={"prompt": prompt}
)
q_p = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type='stuff',
        retriever=vectorstore.as_retriever(),
        memory=conversational_memory,
        get_chat_history=lambda h: h,
        verbose=True
    )

In [ ]:
import gradio as gr
import random
import time

# Chatbot demo with multimodal input (text, markdown, LaTeX, code blocks, image, audio, & video). Plus shows support for streaming text.

def add_text(history, text):
    history = history + [(text, None)]
    return history, gr.update(value="", interactive=True)


def bot(history):
    response = qa(history[-1][0])['result']
    history[-1][1] = ""
    for character in response:
        history[-1][1] += character
        time.sleep(0.05)
        yield history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot([], elem_id="chatbot").style(height=750)

    with gr.Row():
        with gr.Column(scale=0.85):
            txt = gr.Textbox(
                show_label=False,
                placeholder="Enter text and press enter",
            ).style(container=False)
        with gr.Column(scale=0.10):
            submit_btn = gr.Button(
                'Submit',
                variant='primary'
            )
        with gr.Column(scale=0.10):
            clear_btn = gr.Button(
                'Clear',
                variant='stop'
            )
    txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(
        bot, chatbot, chatbot
    )
    submit_btn.click(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(
        bot, chatbot, chatbot
    )
    clear_btn.click(lambda: None, None, chatbot, queue=False)
    txt_msg.then(lambda: gr.update(interactive=True), None, [txt], queue=False)

demo.queue()
if __name__ == "__main__":
    demo.launch()


<ipython-input-38-b2fbae5e3c79>:21: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  chatbot = gr.Chatbot([], elem_id="chatbot").style(height=750)
<ipython-input-38-b2fbae5e3c79>:24: GradioDeprecationWarning: 'scale' value should be an integer. Using 0.85 will cause issues.
  with gr.Column(scale=0.85):
<ipython-input-38-b2fbae5e3c79>:25: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  txt = gr.Textbox(
<ipython-input-38-b2fbae5e3c79>:29: GradioDeprecationWarning: 'scale' value should be an integer. Using 0.1 will cause issues.
  with gr.Column(scale=0.10):
<ipython-input-38-b2fbae5e3c79>:34: GradioDeprecationWarning: 'scale' value should be an integer. Using 0.1 will cause issues.
  with gr.Column(scale=0.10):


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4c8b521d10e5e9e0d4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
